# Football table constructer

#### Students
- Marcus Christiansen Hippe - 49833
- Amund Brenne Reigstad - 50672
- Maximilian  Mohr - 50145

#### Description
Have you ever had csv. files of all the football matches from the 16/17, 17/18 and 18/19 seasons of the Premier League or Ligue 1 and wondered how the table looked in the end? Yes? Perfect! We even added a table that represents the weighted average of all three seasons. Run the script below and and we'll send you a summary right away!

We create a standardised script that can turn all match information of a season of any of the top European football leagues into the correct season table. This includes restructuring the data, calculating the goal difference, points, wins, loses and draws per team and ranking the teams according to the rules of the game.

Furthermore, we aim to compare the performance of the clubs over time and provide detailed information on the best performing club of three seasons. Therefore, we rank the clubs based on their average performance of the previous seasons and pull information on the top-performing club via API.

To run the file you need to download the following packages:
- Pandas
- Numpy
- Requests
- from Urllib.request import Urlopen
- from Bs4 import BeautifulSoup
- Csv
- Re
- smtplib
- email.mime.text
- email.mime.multipart

### Importing the necessary packages:

In [48]:
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from os import getcwd
import csv
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

You can choose between the following football leagues: 

- Premier League (England)
- Ligue one (France)

The program will provide you with information regarding the respective league tables in the follwowing years:
- 2016-17
- 2017-18
- 2018-19

In [49]:
data = input("- If you select Premier league: press enter\n- If you select Ligue one: press space and then enter:")

- If you select Premier league: press enter
- If you select Ligue one: press space and then enter:


In [50]:
def import_file(url):

    download = requests.Session().get(url)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    return(list(cr))


print(data)

if data == "":
    data_16_17 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/16_17_revised.csv")
    data_17_18 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/17_18_revised.csv")
    data_18_19 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/18_19_revised.csv")
    league = "Premier League"

else:
    data_16_17 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/season-1617_ligue1.csv")   
    data_17_18 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/season-1718_ligue1.csv")    
    data_18_19 = import_file("https://raw.githubusercontent.com/marhip97/Introduction-to-programming-Group-Project/main/season-1819_ligue1.csv")    
    league = "Ligue one"


### Organising the data
In the organise_data function, we are changing the format (csv) from the data input to fit our purpose and creating a dictionary with the relevant data points. This is done by splitting the csv data into columns and creating a dictionary with matches as keys and relevant individual match information:

In [51]:
def organise_data(data):
    #data = open(data, "r")
    dict1 = {}
    number = 0
    for i in data:
        if number == 0:
            number +=1
        else:
            Home_team = str(i[1])
            Away_team = str(i[2])
            GH = int(i[3])
            GA = int(i[4])
            YH = int(i[5])
            YA = int(i[6])

            dict1[number] = [Home_team, Away_team, GH, GA, YH, YA]

            number += 1
    return dict1


season_16_17 = organise_data(data_16_17)
season_17_18 = organise_data(data_17_18)
season_18_19 = organise_data(data_18_19)

### Allocate points and goal difference
In the allocate_points_and_goaldiff function, we are creating a dictionary of all the teams in the league (20 in total), by restructuring the previous dictionary with individual matches as keys to a dictionary that with the individual teams as keys.
Simultaneously counting matches, wins, loses and draws per team and calculating goal difference and point allocation:

In [52]:
def allocate_points_and_goaldiff(data, season):
    dict1={}
    for i in data.keys():
        dict1[data[i][0]]={"Games_played":0,"Wins":0, "Losses":0, "Draws":0, "Points":0, "Goal_diff":0, "Yellow_cards":0}
    assert len(dict1)==20
    
    for x in data.keys():
        dict1[data[x][0]]["Games_played"] +=1
        dict1[data[x][1]]["Games_played"] +=1

        if data[x][2] > data[x][3]:
            dict1[data[x][0]]["Wins"] +=1
            dict1[data[x][0]]["Points"] += 3
            dict1[data[x][1]]["Losses"] += 1
        
        if data[x][2] < data[x][3]:
            dict1[data[x][1]]["Wins"] += 1
            dict1[data[x][1]]["Points"] += 3
            dict1[data[x][0]]["Losses"] += 1
        
        if data[x][2] == data[x][3]:
            dict1[data[x][1]]["Draws"] += 1
            dict1[data[x][1]]["Points"] += 1
            dict1[data[x][0]]["Points"] += 1
            dict1[data[x][0]]["Draws"] += 1
            
        dict1[data[x][0]]["Goal_diff"] += (data[x][2] - data[x][3])
        dict1[data[x][1]]["Goal_diff"] += (data[x][3] - data[x][2])
        
        dict1[data[x][0]]["Yellow_cards"] += data[x][4]
        dict1[data[x][1]]["Yellow_cards"] += data[x][5]
            
    return dict1      
    
    
allocation_season_16_17= allocate_points_and_goaldiff(season_16_17, "16/17")
allocation_season_17_18= allocate_points_and_goaldiff(season_17_18, "17_18")
allocation_season_18_19= allocate_points_and_goaldiff(season_18_19, "18_19")

### Ranking of the teams
In the rank_teams function we are restructuring the dictionary to sort the teams according to the criteria of a football table:

In [53]:
 def rank_teams(data):
    points= {}
    for i in data.keys():
        points[i]=data[i]['Points']
    ls = points.values(); ls = list(ls);ls.sort(reverse=True)
    ranked_dict = {} 
    for i in ls: 
        for p in points:
            if i == points[p]:
                ranked_dict[p]=i
    equal_points = []
    for i in ranked_dict.keys():
        n = 0    
        for p in ls:
            if ranked_dict[i]==p:
                n+=1
        if n >=2:
            equal_points.append(i)
    if equal_points == []:
        return(ranked_dict)
    else:
        goal_diff = {}
        for i in equal_points:
            goal_diff[i] = data[i]["Goal_diff"]
        gd = list(goal_diff.values())
        gd.sort(reverse=True)
        w_gd ={}
        for i in gd: 
            for p in goal_diff:
                if i == goal_diff[p]:
                    w_gd[p]=i
        n=0
        for i in w_gd:
            ranked_dict[i]+=n
            n-=0.01
        ls = ranked_dict.values(); ls = list(ls);ls.sort(reverse=True)
        new_ranked_dict={}
        for i in ls: 
            for p in ranked_dict:
                if i == ranked_dict[p]:
                    new_ranked_dict[p]=i
        for i in new_ranked_dict:
            new_ranked_dict[i]=data[i]
        return (new_ranked_dict)

    
table1_season_16_17 = rank_teams(allocation_season_16_17)  
table1_season_17_18 = rank_teams(allocation_season_17_18)
table1_season_18_19 = rank_teams(allocation_season_18_19)
merge_dictionary_list = [table1_season_16_17, table1_season_17_18, table1_season_18_19]

### Comparing the seasons
In the compare_seasons function we are finding the wighted average of all the three seasons: 2016-17, 2017-18 and 2018-19.

In [54]:
def compare_seasons(data):
    dict1 = {}
    for i in data:
        for x in i.keys():
            dict1[x]={"Games_played":0, 'Wins': 0, 'Losses': 0, 'Draws': 0, 'Points': 0, 'Goal_diff': 0, 'Yellow_cards': 0}
    
    number_of_iterations = 0
    for i in data:
        number_of_iterations += 1
        for x in i.keys():
            dict1[x]["Games_played"] += i[x]["Games_played"]
            dict1[x]['Wins'] += i[x]["Wins"]
            dict1[x]['Losses'] += i[x]["Losses"]
            dict1[x]['Draws'] += i[x]["Draws"]
            dict1[x]['Points'] += i[x]["Points"]
            dict1[x]['Goal_diff'] += i[x]["Goal_diff"]
            dict1[x]['Yellow_cards'] += i[x]["Yellow_cards"]
            
    for i in dict1:
        dict1[i]['Wins'] = int((dict1[i]["Wins"]/dict1[i]["Games_played"])*38)
        dict1[i]['Losses'] = int((dict1[i]["Losses"]/dict1[i]["Games_played"])*38)
        dict1[i]['Draws'] = int((dict1[i]["Draws"]/dict1[i]["Games_played"])*38)
        dict1[i]['Points'] = int((dict1[i]["Points"]/dict1[i]["Games_played"])*38)
        dict1[i]['Goal_diff'] = int((dict1[i]["Goal_diff"]/dict1[i]["Games_played"])*38)
        dict1[i]['Yellow_cards'] = int((dict1[i]["Yellow_cards"]/dict1[i]["Games_played"])*38)
    
    aggregate_table = rank_teams(dict1)
    return aggregate_table
        
merged_table = compare_seasons(merge_dictionary_list)

### Visualisation of the data
In the visualise_data function we are taking the dictionary and convert it into a visual league table. 

In [55]:
def visualise_table(data, season):
    df = pd.DataFrame(data).T
    df["Rank"] = np.arange(1, len(df)+1)
    df["Team"] = df.index
    df.set_index("Rank", inplace = True)
    df = df[['Team','Games_played', 'Wins', 'Losses', 'Draws','Goal_diff','Points']]
    print("\n---------------------------------------------------------------------------")
    print("------------------- {} table {} season -------------------".format(league, season))
    print("---------------------------------------------------------------------------")
    print(df)

### Summary of the winner team
Here we make a function which gives you a summary about the winning team. Firstly, we use a Wikipedia API to search for the winner on the site. From the search results we find the URL for the team's Wikipedia page and scrape certain information from that. Secondly, we use the information to give a summary. This code should in general work for most of the teams, however some variations in the HTML code might differ if you input a team that have a different HTML setup for their Wikipedia site. 

In [56]:
def summarise():
    URL = 'https://en.wikipedia.org/w/api.php'
    teams = list(merged_table.keys()) ; winner = teams[0]
    PARAMS = {
        "action": "opensearch",
        "format": "json",
        "maxlag": "3",
        "search": winner}
    r = requests.get(url = URL, params = PARAMS)
    data = r.json()
    url1 = data[3][0]
    source = urlopen(url1).read()
    soup = BeautifulSoup(source,'lxml')
    text = []
    for i in soup.find_all("tr"):
        v = i.text
        text.append(v)
    for i in range(len(text)):
        text[i] = re.sub(r"([A-Z])", r" \1", str(text[i]))
        text[i] = re.sub('(\d+(\.\d+)?)', r' \1 ', str(text[i]))
        match1 = re.match(" Manager", str(text[i]))
        match2 = re.match(" Short name", str(text[i]))
        match3 = re.match(" Full name", str(text[i]))
        match4 = re.match(" Ground", str(text[i]))
        match5 = re.match(" 2020 – 21  Premier  League", str(text[i]))
        match6 = re.match(" Head coach", str(text[i]))
        match7 = re.match(" 2020 – 21", str(text[i]))

        if match1:
            manager = text[i][len(" Manager")+1:len(str(text[i]))]
            manager = manager.replace("  "," ")
            manager = re.sub("[\(\[].*?[\)\]]", "", manager)
            manager = manager.replace("Carrick ","Carrick")

        if match2:
            short_name = text[i][len(" Short name")+1:len(str(text[i]))]
            short_name = short_name.replace(","," or")
            short_name = short_name.replace("/",",")
            short_name = short_name.replace("Utd","Utd or")
            short_name = re.sub("[\(\[].*?[\)\]]", "", short_name)
            short_name = short_name.replace("  "," ")
        if match3:
            full_name = text[i][len(" Full name")+1:len(str(text[i]))]
            full_name = full_name.replace("  "," ")
        if match4:
            ground = text[i][len(" Ground")+1:len(str(text[i]))]
            ground = ground.replace("  "," ")
            ground = ground.replace("\n",",")
            ground = ground[2:len(ground)-1]
        if match5:
            _20_21 = text[i][len(" 2020 – 21  Premier  League")+3:len(str(text[i]))]
            _20_21 = _20_21.replace("  "," ")
            _20_21 = _20_21.replace("20 ","20")
        if match6:
            head_coach = text[i][len(" Head coach")+1:len(str(text[i]))]
            head_coach = head_coach.replace("  "," ")
            head_coach = re.sub("[\(\[].*?[\)\]]", "", head_coach)
        if match7:
            s_20_21 = text[i][len(" 2020 – 21")+14:len(str(text[i]))]
            s_20_21 = s_20_21.replace("  "," ")
            s_20_21 = s_20_21.replace(" nd","nd")
    if league == "Ligue one": 
        ground = ground.split(",")
        ground = ground[0]
    else:
        ground = list(ground.split(","))
        ground = ground[3]
    try:
        manager
    except NameError:
        manager = head_coach
    try:
        _20_21
    except NameError:
        _20_21 = s_20_21
    return(f"The winner of the weighted average of the seasons 16/17, 17/18 and 18/19 is {full_name}.\nIn daily speech, the club is often referred to as {short_name}, and is led by manager {manager}.\nThe team play their home games at {ground}.\nIn the season 20/21 they placed {_20_21}.\n\nIf you want to read more about the club you can visit their wikipedia page: {url1}")
summarised_data = summarise()

### Sending e-mail function
Here we use the in-built Simple Mail Transfer Protocol (SMTP) module. We then created a gmail adress which less secure settings, so that we could use the following code below. We also added a version which, if the sender cant decode HTML, will get a plain text email instead. 

In [57]:
def send_mail(email):
    gmail_user = 'introprogramming17@gmail.com'
    gmail_password = 'Pythonisfun1'
    sent_from = gmail_user
    to = email
    message = MIMEMultipart("alternative")
    message["Subject"] = "{} summary".format(league)
    message["From"] = gmail_user
    message["To"] = to
    text = """\
    Hi,
    Here is your summary of the three seasons:
    {}
    Have a nice day!""".format(summarised_data.replace("\n",""),)
    html = """\
    <html>
      <body>
        <p>Hi,<br>
           Here is your summary of the three seasons:<br>
           {} <br>
           Have a good day!
        </p>
      </body>
    </html>
    """.format(summarised_data.replace("\n","<br>"),)
    part1 = MIMEText(text, "plain")
    part2 = MIMEText(html, "html")
    message.attach(part1)
    message.attach(part2)
    try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, message.as_string())
        server.close()

        print ('Email sent!')
    except:
        print ('Something went wrong...')


In [58]:
# Visualising all the tables. 
visualise_table(table1_season_16_17, "2016-17")
visualise_table(table1_season_17_18, "2017-18")
visualise_table(table1_season_18_19, "2018-19")
print("\n\n")
visualise_table(merged_table, "2016-19 comparable")
print()
print(summarise())


---------------------------------------------------------------------------
------------------- Premier League table 2016-17 season -------------------
---------------------------------------------------------------------------
                Team  Games_played  Wins  Losses  Draws  Goal_diff  Points
Rank                                                                      
1            Chelsea            38    30       5      3         52      93
2          Tottenham            38    26       4      8         60      86
3           Man City            38    23       6      9         41      78
4          Liverpool            38    22       6     10         36      76
5            Arsenal            38    23       9      6         33      75
6         Man United            38    18       5     15         25      69
7            Everton            38    17      11     10         18      61
8        Southampton            38    12      16     10         -7      46
9        Bournemouth 

If you want to get an e-mail with all the information provided, please type in your e-mail adress:

- If do not want an email, please press enter to close the program. 

In [59]:
x = True
while(x): 
    e_mail = input("Please type in the E-mail address: ")
    if e_mail == "":
        x = False
        print("Have a good day!")
    else:
        if "@" not in e_mail:
            print("Make sure the e-mail adress is correct. please try again.")
        else:
            x = False
            send_mail(e_mail)

Please type in the E-mail address: 
Have a good day!
